# **Import**

In [ ]:
import time
from functools import wraps
from typing import Any, Callable
import sys
sys.path.append('..')

from app.db.services import MockSyncService

# **Initialize**

In [ ]:
mock_service = MockSyncService()

In [ ]:
def timing(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Measure and print the execution time of a function.

    Parameters
    ----------
    func : Callable
        The function to measure.

    Returns
    -------
    Callable
        A wrapper that prints execution time before returning the result.
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__qualname__} took {(end - start) * 1000:.2f} ms")
        return result
    return wrapper

# **Wrap instance methods with timing decorator**

In [ ]:
original_run_prediction = mock_service.run_prediction
mock_service.run_prediction = timing(original_run_prediction)

original_get_user = mock_service.get_user
mock_service.get_user = timing(original_get_user)

original_update_user = mock_service.update_user
mock_service.update_user = timing(original_update_user)

# **Test**

## **I. Test caching of `run_prediction()` method**

In [ ]:
mock_service.run_prediction(model_id="id", input_data=[3,1,4,1,5])

In [ ]:
mock_service.run_prediction(model_id="id", input_data=[3,1,4,1,5])

In [ ]:
# different inputs cause slow execution
mock_service.run_prediction(model_id="id", input_data=[3,2,4,1,5])

## **II. Test caching of `get_user()` method**

In [ ]:
mock_service.get_user(user_id=7)

In [ ]:
mock_service.get_user(user_id=7)

In [ ]:
# after waiting for REDIS_TTL_FAST to expire and calling the method again with the same inputs, it executes slowly
mock_service.get_user(user_id=7)

## **III. Test invalidating cache of `get_user()` method**

In [ ]:
mock_service.update_user(user_id=7)

In [ ]:
# after invalidating the cache of `get_user()`, calling it again with the same inputs triggers slow execution again
mock_service.get_user(user_id=7)